In [3]:
import os
import redshift_utils as rs
import pandas as pd
import time
import sagemaker
import boto3
from skrub import TableReport
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [4]:
bucket = "edcs3.prod.biads"
boto_session = boto3.session.Session(region_name="eu-west-1")
sess = sagemaker.session.Session(boto_session, default_bucket=bucket)
region = sess.boto_session.region_name
account = sess.boto_session.client('sts').get_caller_identity()['Account']
role = sagemaker.get_execution_role()
ssm_client = boto3.client('ssm')
kms_key = ssm_client.get_parameter(Name='/prod/sagemaker/BIADS/sloth/KMS_KEY_ID', WithDecryption=True)["Parameter"]["Value"] 

In [5]:
def load_data(params, sql):
    sloth_credentials = rs.get_sloth_credentials()
    con = rs.connect_to_redshift(sloth_credentials)
    data = rs.execute_sql(
        RSconn=con,
        sql=sql,
        input_type="file",
        params=params
    )
    return data

In [6]:
# Create prep table
params = {'@iam': 'arn:aws:iam::765819017647:role/rs-edcs3.prod.marketing',
          '@kms': kms_key, 
         }
data = load_data(params=params, sql='../data/cards_sim.sql')
pd.set_option('display.max_columns', None)  # Show all columns
df = pd.DataFrame(data)
df.head(50)

In [ ]:
from skrub import TableReport
# total_summary = df[['total_stakes', 'total_payout']].sum()
# print(total_summary)

TableReport(df)

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

color_config = {
    'PP': {'main': '#086315', 'top': '#30953e'}, 
    'BF': {'main': '#ff7f0e', 'top': '#ffbb78'}, 
    'SBG': {'main': '#692773', 'top': '#33347c'}
}

plot_title = "Total Stakes & Payout by Brand and In-Play/ Pre-Match Status"

# --- DATA PREPARATION ---
# First, let's check what values exist in in_play_yn
print("Unique values in in_play_yn column:", df['in_play_yn'].unique())
print("Value counts for in_play_yn:")
print(df['in_play_yn'].value_counts())

# Group by brand and in_play_yn, sum total_stakes and total_payout
grouped_df = df.groupby(['brand', 'in_play_yn']).agg({
    'total_stakes': 'sum',
    'total_payout': 'sum'
}).reset_index()

# Calculate margin (difference between stakes and payout)
grouped_df['total_margin'] = grouped_df['total_stakes'] - grouped_df['total_payout']

print("\nGrouped data:")
print(grouped_df)

# --- PLOTTING SETUP ---
brands = sorted(grouped_df['brand'].unique())
inplay_statuses = sorted(grouped_df['in_play_yn'].unique())
num_brands = len(brands)
num_inplay = len(inplay_statuses)

# Create x-axis positions for each brand-inplay combination
x_labels = []
x_indices = []
current_x = 0

for brand in brands:
    for inplay in inplay_statuses:
        x_labels.append(f"{brand}\n({'Inplay' if (inplay == 'Y') else 'Pre-match'})")
        x_indices.append(current_x)
        current_x += 1
    current_x += 0.5  # Add spacing between brands

x_indices = np.array(x_indices)

# Dynamic figure width based on the number of combinations
fig, ax = plt.subplots(figsize=(max(12, len(x_indices) * 1.2), 8))

# --- CREATE BARS ---
bar_width = 0.6

for i, (brand, inplay) in enumerate([(b, ip) for b in brands for ip in inplay_statuses]):
    # Get data for the current brand-inplay combination
    brand_inplay_data = grouped_df[
        (grouped_df['brand'] == brand) & 
        (grouped_df['in_play_yn'] == inplay)
    ]
    
    if not brand_inplay_data.empty:
        stakes = brand_inplay_data['total_stakes'].iloc[0]
        payout = brand_inplay_data['total_payout'].iloc[0]
        margin = brand_inplay_data['total_margin'].iloc[0]
        
        # Plot total_payout (the 'main' part of the bar)
        ax.bar(x_indices[i], payout, bar_width,
               color=color_config.get(brand, {}).get('main', '#808080'))
        
        # Plot total_margin on top of total_payout (to show total stakes)
        ax.bar(x_indices[i], margin, bar_width,
               bottom=payout,
               color=color_config.get(brand, {}).get('top', '#A9A9A9'))

# --- LABELS, TITLE, AND TICKS ---
ax.set_ylabel('Amount (£)', fontsize=14)
ax.set_xlabel('Brand & Match Type', fontsize=14)
ax.set_title(plot_title, fontsize=16, pad=20)
ax.set_xticks(x_indices)
ax.set_xticklabels(x_labels, rotation=45, ha="right")

# --- FORMATTING ---
# Currency formatter for the y-axis
def currency_formatter_bmt(x, pos):
    if x >= 1_000_000_000:
        return f'£{x*1e-9:1.1f}B'
    if x >= 1_000_000:
        return f'£{x*1e-6:1.1f}M'
    if x >= 1_000:
        return f'£{x*1e-3:1.0f}K'
    return f'£{x:1.0f}'

ax.yaxis.set_major_formatter(mticker.FuncFormatter(currency_formatter_bmt))

# --- LEGEND ---
# Create legend for brands and stack meaning
from matplotlib.patches import Patch
legend_elements = []

# Brand colors
for brand in brands:
    legend_elements.append(
        Patch(facecolor=color_config.get(brand, {}).get('main', '#808080'), 
              edgecolor='none', label=f'{brand}')
    )

ax.legend(handles=legend_elements, title='Brand & Components', 
          bbox_to_anchor=(1.03, 1), loc='upper left', borderaxespad=0.)

# --- STYLING AND LAYOUT ---
plt.grid(True, which='major', axis='y', linestyle=':', linewidth=0.7, alpha=0.6)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Adjust layout to make room for the legend
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()

# --- OPTIONAL: Print summary statistics ---
print("\nSummary by Brand and Inplay Status:")
print("=" * 50)
for brand in brands:
    print(f"\n{brand}:")
    brand_data = grouped_df[grouped_df['brand'] == brand]
    for _, row in brand_data.iterrows():
        inplay_text = "In-Play" if row['in_play_yn'] == 'Y' else "Pre-Match"
        print(f"  {inplay_text}:")
        print(f"    Total Stakes: £{row['total_stakes']:,.2f}")
        print(f"    Total Payout: £{row['total_payout']:,.2f}")

Unique values in in_play_yn column: ['N' 'Y']
Value counts for in_play_yn:
in_play_yn
N    1544971
Y     146203
Name: count, dtype: int64


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:21                                                                                   │
│                                                                                                  │
│    18 print(df['in_play_yn'].value_counts())                                                     │
│    19                                                                                            │
│    20 # Group by brand and in_play_yn, sum total_stakes and total_payout                         │
│ ❱  21 grouped_df = df.groupby(['brand', 'in_play_yn']).agg({                                     │
│    22 │   'total_stakes': 'sum',                                                                 │
│    23 │   'total_payout': 'sum'                                                                  │
│    24 }).reset_index()                                                                           │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/cenv/lib/python3.10/site-packages/pandas/core/groupby/generic.p │
│ y:1432 in aggregate                                                                              │
│                                                                                                  │
│   1429 │   │   │   kwargs["engine_kwargs"] = engine_kwargs                                       │
│   1430 │   │                                                                                     │
│   1431 │   │   op = GroupByApply(self, func, args=args, kwargs=kwargs)                           │
│ ❱ 1432 │   │   result = op.agg()                                                                 │
│   1433 │   │   if not is_dict_like(func) and result is not None:                                 │
│   1434 │   │   │   # GH #52849                                                                   │
│   1435 │   │   │   if not self.as_index and is_list_like(func):                                  │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/cenv/lib/python3.10/site-packages/pandas/core/apply.py:190 in   │
│ agg                                                                                              │
│                                                                                                  │
│    187 │   │   │   return self.apply_str()                                                       │
│    188 │   │                                                                                     │
│    189 │   │   if is_dict_like(func):                                                            │
│ ❱  190 │   │   │   return self.agg_dict_like()                                                   │
│    191 │   │   elif is_list_like(func):                                                          │
│    192 │   │   │   # we require a list, but not a 'str'                                          │
│    193 │   │   │   return self.agg_list_like()                                                   │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/cenv/lib/python3.10/site-packages/pandas/core/apply.py:423 in   │
│ agg_dict_like                                                                                    │
│                                                                                                  │
│    420 │   │   -------                                                                           │
│    421 │   │   Result of aggregation.                                                            │
│    422 │   │   """                                                                               │
│ ❱  423 │   │   return self.agg_or_apply_dict_like(op_name="

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

plot_title = "Total Payout by Player Card (In-Play vs Pre-Match)"

# --- DATA PREPARATION ---
# Group by sportex_selection_name and in_play_yn, sum total_payout
grouped_df = df.groupby(['sportex_selection_name', 'in_play_yn']).agg({
    'total_payout': 'sum'
}).reset_index()

# Pivot to get in-play and pre-match as separate columns
pivot_df = grouped_df.pivot(index='sportex_selection_name', 
                           columns='in_play_yn', 
                           values='total_payout').fillna(0)

# Handle cases where some sports might not have both Y and N values
if 'Y' not in pivot_df.columns:
    pivot_df['Y'] = 0
if 'N' not in pivot_df.columns:
    pivot_df['N'] = 0

# Calculate total payout for sorting
pivot_df['total_payout'] = pivot_df['Y'] + pivot_df['N']

# Sort by total payout (largest to smallest)
pivot_df = pivot_df.sort_values('total_payout', ascending=True)  # ascending=True for horizontal bars

print("Top 10 Sport Selections by Total Payout:")
print(pivot_df[['Y', 'N', 'total_payout']].tail(10))

# --- PLOTTING SETUP ---
sport_selections = pivot_df.index.tolist()
num_selections = len(sport_selections)

# Use viridis colormap - get two distinct colors
viridis_colors = plt.cm.viridis([0.2, 0.8])  # Get colors from viridis palette
in_play_color = viridis_colors[0]      # Darker viridis color for in-play
pre_match_color = viridis_colors[1]    # Lighter viridis color for pre-match

# Dynamic figure height based on number of selections
fig, ax = plt.subplots(figsize=(12, max(8, num_selections * 0.4)))

# --- CREATE BARS ---
y_positions = np.arange(num_selections)

# Create horizontal stacked bars
pre_match_values = pivot_df['N'].values
in_play_values = pivot_df['Y'].values

# Plot pre-match (left part of bar)
bars1 = ax.barh(y_positions, pre_match_values, 
                color=pre_match_color, 
                label='Pre-Match (N)',
                alpha=0.8)

# Plot in-play (right part of bar, starting where pre-match ends)
bars2 = ax.barh(y_positions, in_play_values, 
                left=pre_match_values,
                color=in_play_color, 
                label='In-Play (Y)',
                alpha=0.8)

# --- LABELS, TITLE, AND TICKS ---
ax.set_xlabel('Total Payout (£)', fontsize=14)
ax.set_ylabel('Sport Selection', fontsize=14)
ax.set_title(plot_title, fontsize=16, pad=20)
ax.set_yticks(y_positions)
ax.set_yticklabels(sport_selections)

# --- FORMATTING ---
# Currency formatter for the x-axis
def currency_formatter_bmt(x, pos):
    if x >= 1_000_000_000:
        return f'£{x*1e-9:1.1f}B'
    if x >= 1_000_000:
        return f'£{x*1e-6:1.1f}M'
    if x >= 1_000:
        return f'£{x*1e-3:1.0f}K'
    return f'£{x:1.0f}'

ax.xaxis.set_major_formatter(mticker.FuncFormatter(currency_formatter_bmt))

# --- LEGEND ---
ax.legend(title='Bet Type', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)

# --- STYLING AND LAYOUT ---
plt.grid(True, which='major', axis='x', linestyle=':', linewidth=0.7, alpha=0.6)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Adjust layout to make room for labels and legend
plt.tight_layout(rect=[0, 0, 0.85, 1])

# --- ADD VALUE LABELS ON BARS (OPTIONAL) ---
# Add percentage labels showing split
for i, (sport_selection, row) in enumerate(pivot_df.iterrows()):
    total = row['total_payout']
    pre_match_pct = (row['N'] / total * 100) if total > 0 else 0
    in_play_pct = (row['Y'] / total * 100) if total > 0 else 0
    
    # Only add labels if the values are significant enough to be readable
    if total > 0:
        # Add total value at the end of each bar
        ax.text(total + total*0.01, i, f'£{total:,.0f}', 
                va='center', ha='left', fontsize=9, alpha=0.8)
        
        # Add percentage splits inside bars (if segments are large enough)
        if pre_match_pct > 5:  # Only show if segment is >5%
            ax.text(row['N']/2, i, f'{pre_match_pct:.0f}%', 
                    va='center', ha='center', fontsize=8, color='white', weight='bold')
        if in_play_pct > 5:  # Only show if segment is >5%
            ax.text(row['N'] + row['Y']/2, i, f'{in_play_pct:.0f}%', 
                    va='center', ha='center', fontsize=8, color='white', weight='bold')

plt.show()

# --- SUMMARY STATISTICS ---
print(f"\nSummary Statistics:")
print(f"Total number of sport selections: {len(sport_selections)}")
print(f"Total payout across all selections: £{pivot_df['total_payout'].sum():,.2f}")
print(f"Total pre-match payout: £{pivot_df['N'].sum():,.2f}")
print(f"Total in-play payout: £{pivot_df['Y'].sum():,.2f}")
print(f"Overall in-play percentage: {(pivot_df['Y'].sum() / pivot_df['total_payout'].sum() * 100):.1f}%")

print(f"\nTop 5 Sport Selections by Total Payout:")
top_5 = pivot_df.tail(5)[['Y', 'N', 'total_payout']].iloc[::-1]  # Reverse to show highest first
for sport, row in top_5.iterrows():
    in_play_pct = (row['Y'] / row['total_payout'] * 100) if row['total_payout'] > 0 else 0
    print(f"  {sport}: £{row['total_payout']:,.0f} ({in_play_pct:.1f}% in-play)")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│     7                                                                                            │
│     8 # --- DATA PREPARATION ---                                                                 │
│     9 # Group by sportex_selection_name and in_play_yn, sum total_payout                         │
│ ❱  10 grouped_df = df.groupby(['sportex_selection_name', 'in_play_yn']).agg({                    │
│    11 │   'total_payout': 'sum'                                                                  │
│    12 }).reset_index()                                                                           │
│    13                                                                                            │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/cenv/lib/python3.10/site-packages/pandas/core/groupby/generic.p │
│ y:1432 in aggregate                                                                              │
│                                                                                                  │
│   1429 │   │   │   kwargs["engine_kwargs"] = engine_kwargs                                       │
│   1430 │   │                                                                                     │
│   1431 │   │   op = GroupByApply(self, func, args=args, kwargs=kwargs)                           │
│ ❱ 1432 │   │   result = op.agg()                                                                 │
│   1433 │   │   if not is_dict_like(func) and result is not None:                                 │
│   1434 │   │   │   # GH #52849                                                                   │
│   1435 │   │   │   if not self.as_index and is_list_like(func):                                  │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/cenv/lib/python3.10/site-packages/pandas/core/apply.py:190 in   │
│ agg                                                                                              │
│                                                                                                  │
│    187 │   │   │   return self.apply_str()                                                       │
│    188 │   │                                                                                     │
│    189 │   │   if is_dict_like(func):                                                            │
│ ❱  190 │   │   │   return self.agg_dict_like()                                                   │
│    191 │   │   elif is_list_like(func):                                                          │
│    192 │   │   │   # we require a list, but not a 'str'                                          │
│    193 │   │   │   return self.agg_list_like()                                                   │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/cenv/lib/python3.10/site-packages/pandas/core/apply.py:423 in   │
│ agg_dict_like                                                                                    │
│                                                                                                  │
│    420 │   │   -------                                                                           │
│    421 │   │   Result of aggregation.                                                            │
│    422 │   │   """                                                                               │
│ ❱  423 │   │   return self.agg_or_apply_dict_like(op_name="